In [144]:
import gurobipy as gp
from gurobipy import GRB

In [145]:
network, distances = gp.multidict({
    (1, 2): 25,
    (1, 3): 20,
    (2, 3): 3,
    (2, 4): 5,
    (2, 6): 14,
    (3, 2): 3,
    (3, 5): 6,
    (4, 2): 5,
    (4, 5): 4,
    (4, 6): 4,
    (5, 3): 6,
    (5, 4): 4,
    (5, 6): 7,
})

transshipment = [2, 3, 4, 5]

In [146]:
m = gp.Model('shortest_path')

In [147]:
path = m.addVars(network, vtype=GRB.CONTINUOUS, name='X')

In [148]:
Z = path.prod(distances)  # Sum((i,j), d[i,j] * x[i,j] for i in I for j in J)
m.ModelSense = GRB.MINIMIZE
m.setObjective(Z)

In [149]:
source_constraint = m.addConstr(
            path.sum(1, '*') == 1,
            name='source'
)

transshipment_constraints = m.addConstrs( 
                                (path.sum(t, '*') - path.sum('*', t) == 0 for t in transshipment),
    name='transshipment'
)

sink_constraint = m.addConstr(
    path.sum('*', 6) == 1,
    name='sink'
)

In [150]:
m.optimize()

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 6 rows, 13 columns and 26 nonzeros
Model fingerprint: 0x16357ff9
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [3e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 1 rows and 2 columns
Presolve time: 0.01s
Presolved: 5 rows, 11 columns, 22 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.4000000e+01   2.000000e+00   0.000000e+00      0s
       3    3.2000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.200000000e+01


In [151]:
print(f'Objective Value {m.ObjVal} miles')

Objective Value 32.0 miles


In [152]:
for v in m.getVars():
    if v.x == 1:
        print(v.varName, v.x)

X[1,3] 1.0
X[2,4] 1.0
X[3,2] 1.0
X[4,6] 1.0


In [153]:
m.getConstrs()

[<gurobi.Constr source>,
 <gurobi.Constr transshipment[2]>,
 <gurobi.Constr transshipment[3]>,
 <gurobi.Constr transshipment[4]>,
 <gurobi.Constr transshipment[5]>,
 <gurobi.Constr sink>]